In [2]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import copy

import random


In [3]:
corpus = []
f = open('alice_in_wonderland.txt','r')
while(1):
    line =  f.readline()
    if len(line) == 0: break
    corpus.extend(line.split())
        
f.close()
corpus = ' '.join(corpus)

def clean_word(word):
    word = word.lower()
    for punctuation in ['"',"'",'.',',','-','?','!',';',':','—','(',')','[',']']:
        word = word.split(punctuation)[0]
    return word



corpus = [clean_word(word) for word in corpus.split()]
corpus = [word for word in corpus if len(word) > 0]
print(corpus[:25])
D = len(corpus)
print('corpus len: ',D)

['alice', 'adventures', 'in', 'wonderland', 'by', 'lewis', 'carroll', 'the', 'millennium', 'fulcrum', 'edition', '3', 'contents', 'chapter', 'i', 'down', 'the', 'rabbit', 'chapter', 'ii', 'the', 'pool', 'of', 'tears', 'chapter']
corpus len:  25320


In [4]:
tokenize = {}
wordlist = []
token = 0
for word in corpus:
    if word not in tokenize.keys():
        tokenize[word] = token
        wordlist.append(word)
        token += 1
    
V = len(wordlist)
print('word list size (number of distinct words): ', V)


word list size (number of distinct words):  2501


In [5]:
# bin how many times a word follows another word
counts_2gram = np.zeros((V,V))
for i in range(1,len(corpus)):
    token_i = tokenize[corpus[i]]
    token_im1 = tokenize[corpus[i-1]]
    counts_2gram[token_i,token_im1] += 1
print(counts_2gram)
    

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [9. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
#past word as feature

#posterior is just counts normalized across columns. 
posterior_1word = counts_2gram / np.outer(np.dot(counts_2gram,np.ones(V)),np.ones(V))


    
#prior is just base frequency of words. 
prior = np.zeros(V)
for word in corpus:
    token = tokenize[word]    
    prior[token] += 1
prior = prior / np.sum(prior)



def get_likelihood_2gram(word):
    
    token = tokenize[word]
    
    
    likelihood = posterior_1word[:,token].T*prior
    #likelihood = posterior_1word[token]*prior
    return(likelihood)
def pred_2gram(word):
    likelihood = get_likelihood_2gram(word)
    i = np.argmax(likelihood)
    return(wordlist[i], likelihood[i])
print(pred_2gram('alice'))
print(pred_2gram('the'))
print(pred_2gram('cheshire'))
print(pred_2gram('mock'))


    

('said', 0.005374061880700568)
('said', 0.0023591151671500023)
('the', 0.031852290679304895)
('the', 0.06035170865552506)


In [7]:
#get accuracy of bigram classifier
correct = 0.
for k in range(1,D):
    word = corpus[k]
    prev_word = corpus[k-1]
    pred_word = pred_2gram(prev_word)[0]
    if word == pred_word: correct += 1
print(correct / (D-1))



0.021999289071448318


In [8]:
#past 2 words as features


# bin how many times a word follows another word
counts_anygram = np.zeros((V,V,30))
posterior_anyword = np.zeros((V,V,30))
for gram in range(1,30):
    
    for i in range(gram,len(corpus)):
        token_i = tokenize[corpus[i]]
        token_im1 = tokenize[corpus[i-gram]]
        counts_anygram[token_i,token_im1,gram] += 1
    posterior_anyword[:,:,gram] = counts_anygram[:,:,gram] / np.outer(np.dot(counts_anygram[:,:,gram],np.ones(V)),np.ones(V))


/tmp/ipykernel_26713/1036792518.py:13: RuntimeWarning: invalid value encountered in true_divide
  posterior_anyword[:,:,gram] = counts_anygram[:,:,gram] / np.outer(np.dot(counts_anygram[:,:,gram],np.ones(V)),np.ones(V))


In [9]:


def get_likelihood_anygram(x_list):
    x_list = copy.deepcopy(x_list)
    x_list.reverse()
    likelihood = prior+0.
    for gram_m1 in range(len(x_list)):
        x = x_list[gram_m1]
        tokenx = tokenize[x]
        posterior = posterior_anyword[:, tokenx, gram_m1+1]
        
    
        likelihood = likelihood * posterior
    return likelihood
def pred_anygram(x_list):
    likelihood = get_likelihood_anygram(x_list)
    i = np.argmax(likelihood)
    return wordlist[i], likelihood[i]
print(pred_anygram([ 'falling', 'down', 'a', 'very', 'deep']))
print(pred_anygram(['what', 'an', 'ignorant', 'little']))
print(pred_anygram(['four', 'thousand',]))




('well', 4.702797636375411e-11)
('girl', 1.8513033175355449e-06)
('miles', 1.3164823591363875e-05)


In [10]:
#get accuracy of anygram classifier
for gram in [3,5,10]:
    correct = 0.
    for k in range(gram,D):
        word = corpus[k]
        prev_words = corpus[k-gram:k]
        pred_word = pred_anygram(prev_words)[0]
        if word == pred_word: 
            correct += 1
    print(correct / (D-1))

0.7499111339310399
0.9400845215055887
0.0


In [11]:
gen_anygram = ['the', 'mad', 'hatter']
for i in range(25):
    word = pred_anygram(gen_anygram[-2:])[0]
    gen_anygram.append(word)
print(' '.join(gen_anygram))

print('\n')




the mad hatter all move one place for a minute or two sobs choked his voice has a timid voice at the top of its mouth open gazing




In [12]:


gen_anygram = ['the', 'mad', 'hatter']
for i in range(25):
    
    likelihood = get_likelihood_anygram(gen_anygram[-2:])
    idx = random.choices(range(V),weights=likelihood)
    
    gen_anygram.append(wordlist[idx[0]])
print(' '.join(gen_anygram))


the mad hatter with his knuckles it was opened by another footman in livery otherwise judging by his garden and alice laughed so much contradicted in her lap
